# Random Graphs Impact Analysis

This notebook analyzes the impact of removing the highest-degree node from different types of random graphs on various centrality metrics.

In [ ]:
# SETUP: workaround to avoid ipynb import errors
import os
import sys

candidate_roots = [
    os.getcwd(),
    os.path.dirname(os.getcwd()),
    os.path.dirname(os.path.dirname(os.getcwd())),
]
for root in candidate_roots:
    src_dir = os.path.join(root, "src")
    if os.path.isdir(src_dir) and src_dir not in sys.path:
        sys.path.append(src_dir)
        break

In [ ]:
import networkx as nx

# define centrality metrics for the analysis
centrality_functions = {
    "degree": nx.degree_centrality,
    "betweenness": nx.betweenness_centrality,
    "closeness": nx.closeness_centrality,
    "eigenvector": lambda G: nx.eigenvector_centrality(G, max_iter=5000),
    # Use numpy variant to avoid error on larger graphs,TODO: results are weird, need to check later
    "katz": lambda G: nx.katz_centrality_numpy(G, alpha=0.01, beta=1.0),
}

# define graph sizes
sizes = [10, 100, 1000]

# define graph types
graph_types = [
    "erdos_renyi", 
    "barabasi_albert", 
    "watts_strogatz"
]

In [ ]:
from graph_build_utlis import make_graph, highest_degree_node
from node_removal_graph_analyser import get_node_removal_impact

# compute the differences in centrality
results = []
for size in sizes:
    for gtype in graph_types:
        G = make_graph(gtype, size)
        node = highest_degree_node(G)

        for metric_name, metric_fn in centrality_functions.items():
            # TODO: add try catch to avoid plotting calculated errors, likely happening on katz
            elapsed, impact = get_node_removal_impact(G, node, metric_fn)
            results.append({
                "size": size,
                "graph_type": gtype,
                "metric": metric_name,
                "impact": impact,
                "graph": G
            })


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm, colors

# TODO: save plots to results folders

pos_cache = {}

# plot the resulting graphs
for r in results:
    size = r["size"]
    gtype = r["graph_type"]
    metric = r["metric"]
    impact = r["impact"]
    G = r["graph"]
    
    removed_node = highest_degree_node(G)

    # layout cache per (graph_type, size)
    # This ensures graphs of same size and type are shown with nodes in the same positions
    # The seed is just a random number, 42 is used a pop-culture reference
    # The K parameter here is used to optimize node distances, for larger graphs, it should be smaller
    # the pos value will be used in the plot
    key = (gtype, size)
    if key not in pos_cache:
        if size >= 500:
            pos_cache[key] = nx.spring_layout(G, seed=42, k=1 / np.sqrt(size))
        else:
            pos_cache[key] = nx.spring_layout(G, seed=42)
    pos = pos_cache[key]

    # normalize impacts for color mapping
    # outputs norm, norm is a normalization function created based on the max absolute value
    max_abs = max((abs(v) for v in impact.values()), default=0.0)
    if max_abs == 0:
        norm = colors.TwoSlopeNorm(vmin=-1.0, vcenter=0.0, vmax=1.0)
    else:
        norm = colors.TwoSlopeNorm(vmin=-max_abs, vcenter=0.0, vmax=max_abs)

    # build node colors
    # node_colors stores the colors of each nodes, based on the index
    # cmap creates the blue white red color scheme
    # the removed node gets the yellow color
    # the normalized value by norm is passed to the cmap(bwr) that than returns the color
    cmap = plt.get_cmap("bwr")
    node_colors = []
    for n in G.nodes():
        if n == removed_node:
            node_colors.append("yellow")
        else:
            node_colors.append(cmap(norm(impact.get(n, 0.0))))

    # visual parameters by size
    # defined experimentally, based on the used graphs sizes
    if size >= 500:
        node_size = 10
        edge_width = 0.2
    elif size >= 100:
        node_size = 25
        edge_width = 0.3
    else:
        node_size = 200
        edge_width = 0.6

    # create figure
    # ax will be used for the colorbar scale
    # 0.8 is the border thickness, used to diffrrentiate from the white background
    fig, ax = plt.subplots(figsize=(6, 5))
    nx.draw(
        G,
        pos,
        node_color=node_colors,
        node_size=node_size,
        edge_color="lightgrey",
        width=edge_width,
        ax=ax,
        with_labels=False,
        linewidths=0.8,
        edgecolors="black"
    )
    # set label
    ax.set_title(f"{gtype}, n={size}, metric={metric}")

    # colorbar for impact scale
    # creates the color scale on the side of the images, based on the color map and normalized values
    # 0.046 defines the widht of the scale bar in 0,04 the padding to the rest of the plot
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, fraction=0.04, pad=0.05)
    cbar.set_label("Δ centrality")

    # save image to results
    safe_filename = f"{gtype}-{size}-{metric}"
    filepath = f"../../results/random-graphs-impact/images/{safe_filename}.png"
    
    # Save the plot instead of showing it
    plt.savefig(filepath, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()  # Close the figure to free memory


In [ ]:
# TODO: create degrees distributions of graphs and impact on centrality metrics, compare to degree distrubuitions
# what values are expected for certain metrics, 
# for example, closeness normally does not create positive impact, degree shouldn't as well, but does so beacause of normalization